In [ ]:
%%javascript
$('#appmode-leave').hide();
$('#copy-binder-link').hide();
$('#visit-repo-link').hide();

In [ ]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown, Math, Latex, clear_output

# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["equi4"]

The hydrolysis Sucrose into Glucose and Fructose is catalysed by the enzyme Invertase.
\begin{equation}
Sucrose + Invertase + \mathrm{H_2O} \to Glucose + Fructose
\end{equation}
There are however several substances that can inhibit the efficacy of the catalyst

Imagine performing a series of experiments using different initial concentration of Sucrose where you measure the rate of formation of Glucose with. The results of your experiments are affected by the presence of a contaminating substance that interferes with the catalytic reaction. Although you can somewhat control the concentration of the contaminant, you cannot completely eliminate it.

1. Determine whether the contaminating substance inhibits the catalytic reaction and the type of the inhibition mechanism, *e.g.* Competitive, Uncompetitive, Non-competitive or Mixed.

2. Determine the maximum rate achieved by the reaction, $V_{max}$ and the Michaelis constant, $K_M$, in the case you could completely eliminate the contamininat.

### Tips:
- Note that every time you restart the experiment the type of the inhibition mechanism may change.

### Instructions:

- Use the slide bar below to select temperature at which you perform the virtual experiment, 
- Click `Perform measurement` to run the virtual experiment and obtain the result of the experiment,
- Click `Download CSV` to export the complete data set for all the experiments as a CSV file.

In [ ]:
# define path to results.csv file
respath = os.path.join(os.getcwd(), "..", "results.csv")

# delete existing result file and setup rng
if os.path.exists(respath):
    os.remove(respath)

class system:
    def __init__(self, vol=0, conc=0, press=0):
        self.vol = vol
        self.conc = conc
        self.press = press
        self.inhibition = 0
        self.seed = 0
        self.Vm = 0
        self.Km = 0
        self.Ki = 0
        self.Kip= 0

class data:
    def __init__(self, start=-1, error=0, label='none', units='pure', value=0,
                minval=-1, maxval=3, text='none'):
        self.start = start
        self.minval = minval
        self.maxval = maxval
        self.error = error
        self.label = label
        self.units = units
        self.value = value
        self.text = text
    
# Experiment setup (+ hidden paramters)
system = system()
def initialiseExperiment():
    global n
    global system 
    global columns_list
    global scatter
    
    scatter = 0.01
    
    n = []
    columns_list = []

    n.append(len(args)) # number of input adjustable parameters    
    n.append(len(result)) # number of results for the experiment

    for i in range(0, n[0]):
        columns_list.append(f"{args[i].label} [{args[i].units}]")
    for i in range(0, n[1]):
        columns_list.append(f"{result[i].label} [{result[i].units}]")

    # Random number seed
    t = int( time.time() * 1000.0 )
    system.seed = ((t & 0xff000000) >> 24) + ((t & 0x00ff0000) >>  8) +((t & 0x0000ff00) <<  8) +((t & 0x000000ff) << 24)
    random.seed(system.seed)

    # Random inhibition type
    rnd = random.random()
    system.inhibition = int(5 * rnd)
    if (system.inhibition > 4):
        system.inhibition = 4
        
    system.Vm = params["Vm"] * (1 + random.random()/2)
    system.Km = params["Km"] * (1 + random.random()/2)
    system.Ki = system.Km * random.random()
    system.Kip= system.Km * random.random()



In [ ]:
# Adjustable input parameters
def initialiseVariables():
    global logScale
    logScale = True
    global args
    args = []
    args.append(
        data(
            label = "[S]",
            minval = -3,
            maxval = 1,
            start = 0.001,
            units = "mol/L",
            value = 0.
        )
    )

    args.append(
        data(
            label = "[I]",
            minval = -3,
            maxval = 0,
            start = 0.001,
            units = "mol/L",
            value = 0.
        )
    )


# Results
def initialiseResults():
    global result
    result = []
    result.append(
        data(
            label = "Reaction Rate",
            start = 0.,
            error = random.random() / 10.,
            units = "mol/L·min"
        )
    )

def measure():
    concS = float(args[0].text.value)
    concI = float(args[0].text.value)
    
    Vm = system.Vm
    Km = system.Km
    Ki = system.Ki
    Kip= system.Kip
    
    # no inhibition
    a = 1
    ap = 1
    
    # competitive
    if (system.inhibition == 1):
        a  = 1 + concI / Ki
        ap = 1
        adp = 1

    # non-competitive
    elif (system.inhibition == 4):
        a  = 1
        ap = 1 + concI / Ki
        adp = 1
    
    # un-competitive
    elif (system.inhibition == 2):
        a  = 1
        ap = 1
        adp = 1. / (1 + concI / Kip)

    # mixed
    elif (system.inhibition == 3):
        a  = 1 + concI / Ki
        ap = 1
        adp = 1. / (1 + concI / Kip)

    res =  (ap * adp) * Vm * concS / ((a * adp)*Km + concS)
    return res

initialiseVariables()


In [ ]:
out_P = ipw.Output()
out_L = ipw.Output()
out_X = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))
    
def calc(btn):
    out_P.clear_output()
    
    # Measurement result
    result[0].value = measure()
    
    # Random error
    result[0].error = result[0].value * scatter * (0.5 - random.random()) * 2
    
    # Output result
    out_R[0].value = f"{result[0].value + result[0].error:.3e}"

    # Read previous lines
    res = pd.read_csv(respath) 
    
    var_list = []
    for i in range(0, n[0]):
        var_list.append(args[i].text.value)
    for i in range(0, n[1]):
        var_list.append(result[i].value + result[i].error)
    
    # Append result
    res.loc[len(res)] = var_list
    res.to_csv(respath, index=False)
    with out_P:
        display(res.tail(50))

def reset(btn):
    if os.path.exists(respath):
        os.remove(respath)

    initialiseResults()
    initialiseExperiment()
    
    res = pd.DataFrame(columns=columns_list)
    res.to_csv(respath, index=False)
    with out_P:
        out_P.clear_output()
        display(res.tail(1))

    with out_X:
        out_X.clear_output()
        
btn_reset = ipw.Button(description="Restart Laboratory", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)

btn_calc = ipw.Button(description="Perform measurement", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)
# ---

rows = []
reset(btn_reset)

args[0].text = ipw.Text(str(args[0].start))
rows.append(ipw.HBox([ipw.Label('Initial concentration of ' + args[0].label + '   :  '),args[0].text]))

args[1].text = ipw.Text(str(args[1].start))
rows.append(ipw.HBox([ipw.Label('Initial concentration of ' + args[1].label + '   :  '),args[1].text]))

out_R = []
for i in range(0, n[1]):
    out_R.append(ipw.Label(value=""))
    rows.append(ipw.HBox([ipw.Label(value=f"Measured {result[i].label} [{result[i].units}]:",
                                    layout=ipw.Layout(width="250px")),
                            out_R[i]]))

rows.append(ipw.HBox([btn_reset, btn_calc, out_L]))

def calc2(btn):
    random.seed(system.seed)
    rnd = random.random()
    iType = int(4 * rnd) + 1
    
    with out_X:
        out_X.clear_output()
        if (iType == 1):
            display(Markdown(r'Competitive inhibition'))
        elif (iType == 2):
            display(Markdown(r'Un-Competitive inhibition'))
        elif (iType == 3):
            display(Markdown(r'Mixed inhibition'))
        elif (iType == 4):
            display(Markdown(r'Non-Competitive inhibition'))
        else:
            display(Markdown(r'No inhibition'))

        display(Markdown(r'$K_M$ = 'rf'{system.Km:7.5}'))
        display(Markdown(r'$V_{max}$ = 'rf'{system.Ki:7.5}'))
        if (iType == 1) or (iType == 3) or (iType == 4):
            display(Markdown(r'$K_i$ = 'rf'{system.Ki:7.5}'))
        if (iType == 2) or (iType == 3) or (iType == 4):
            display(Markdown(r'$K_i^\prime$ = 'rf'{system.Kip:7.5}'))

display(out_X)

btn_calc2 = ipw.Button(description="Check Inhibition Type", layout=ipw.Layout(width="150px"))
btn_calc2.on_click(calc2)
rows.append(ipw.HBox([btn_calc2]))

rows.append(ipw.HBox([out_P]))
ipw.VBox(rows)


